In [ ]:
import os
import numpy as np
import tensorflow as tf
import zipfile
from tensorflow.keras import layers, models, callbacks, optimizers
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = (299, 299)
learning_rates = [1e-3, 5e-4]
weight_decays   = [0.0, 1e-4]
max_epochs      = 10
patience        = 3

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount= True)


Mounted at /content/drive


Dataset Loading

In [ ]:
zip_path = '/content/drive/MyDrive/MLMA Project/GROUPBY SPLIT/Use This Final Final (22 04 25)/augumented_final.zip'
output_dir = '/content/final_dataset'


os.makedirs(output_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print(f"Unzipped '{zip_path}' to '{output_dir}'") #Unzipping the final augmented dataset

Unzipped '/content/drive/MyDrive/MLMA Project/GROUPBY SPLIT/Use This Final Final (22 04 25)/augumented_final.zip' to '/content/final_dataset'


In [ ]:
train_dir = r"//content/final_dataset/augumented_final/train"
valid_dir = r"/content/final_dataset/augumented_final/valid"
test_dir  = r"/content/final_dataset/augumented_final/test"

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    label_mode="int",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    valid_dir,
    label_mode="int",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    label_mode="int",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)


class_names = train_ds.class_names
num_classes = len(class_names)
print("Classes:", class_names)

#Final train, val and test sets
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

Found 8958 files belonging to 8 classes.
Found 1282 files belonging to 8 classes.
Found 631 files belonging to 8 classes.
Classes: ['A', 'C', 'D', 'G', 'H', 'M', 'N', 'O']


Model training and Evaluation

In [ ]:
# Function to build VGG19 model

def build_vgg19():
    base = tf.keras.applications.VGG19(
        include_top=False,
        weights="imagenet",
        input_shape=(*IMG_SIZE, 3)
    )
    base.trainable = False #Freezing backbone
    x = layers.GlobalAveragePooling2D()(base.output)
    out = layers.Dense(num_classes, activation="softmax")(x) #Modification of head classifier for 8 classes
    return models.Model(base.input, out)


best_val_acc = 0.0
best_model   = None
best_config  = None

for lr in learning_rates: #Grid search over learning rates and weight decays
    for wd in weight_decays:
        print(f"\n Training with lr={lr}, weight_decay={wd}")
        model = build_vgg19()

        optimizer = optimizers.AdamW(learning_rate=lr, weight_decay=wd) #Optimizer

        model.compile(
            optimizer=optimizer,
            loss="sparse_categorical_crossentropy", #Loss function
            metrics=["accuracy"]
        )

        es = callbacks.EarlyStopping( #Early stopping based on val loss
            monitor="val_loss", patience=patience, restore_best_weights=True
        )

        # Training
        history = model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=max_epochs,
            callbacks=[es],
            verbose=2
        )

        val_acc = history.history["val_accuracy"][-1]
        print(f" Finished: val_accuracy = {val_acc:.4f}")

        if val_acc > best_val_acc: #Selecting best configuration based on highest validation accuracy
            best_val_acc = val_acc
            best_model   = tf.keras.models.clone_model(model)
            best_model.set_weights(model.get_weights())
            best_config  = (lr, wd)

print(f"Best config: lr={best_config[0]}, weight_decay={best_config[1]}, val_acc={best_val_acc:.4f}")


 Training with lr=0.001, weight_decay=0.0
80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
280/280 - 24s - 86ms/step - accuracy: 0.4420 - loss: 1.6396 - val_accuracy: 0.5094 - val_loss: 1.4072
Epoch 2/10
280/280 - 9s - 34ms/step - accuracy: 0.5230 - loss: 1.2709 - val_accuracy: 0.5281 - val_loss: 1.3351
Epoch 3/10
280/280 - 9s - 34ms/step - accuracy: 0.5560 - loss: 1.1720 - val_accuracy: 0.5343 - val_loss: 1.2859
Epoch 4/10
280/280 - 9s - 34ms/step - accuracy: 0.5716 - loss: 1.1158 - val_accuracy: 0.5320 - val_loss: 1.3039
Epoch 5/10
280/280 - 9s - 34ms/step - accuracy: 0.5911 - loss: 1.0703 - val_accuracy: 0.5250 - val_loss: 1.3154
Epoch 6/10
280/280 - 9s - 34ms/step - accuracy: 0.5990 - loss: 1.0393 - val_accuracy: 0.5328 - val_loss: 1.2908
 Finished: val_accuracy = 0.5328

 Training with lr=0.001, weight_decay=0.0001
Epoch 1/10
280/280 - 14s - 51ms/step - accuracy: 0.4606 - loss: 1.6143 - val_accuracy: 0.4735 - val_loss: 1.4381
Epoch 2/10
280/280 - 9s - 34ms/step - acc

In [ ]:
# Evaluate the best model on the test set

y_true = []
y_pred = []

for images, labels in test_ds:

    preds = best_model.predict(images) #Predictions
    y_true.extend(labels.numpy()) #Ground truth
    y_pred.extend(np.argmax(preds, axis=1).tolist())

y_true = np.array(y_true)
y_pred = np.array(y_pred)

acc       = accuracy_score(y_true, y_pred) #Accuracy
prec_w    = precision_score(y_true, y_pred, average='weighted', zero_division=0) #Weighted Precision
rec_w     = recall_score(y_true, y_pred, average='weighted', zero_division=0) #Weighted Recall
f1_w      = f1_score(y_true, y_pred, average='weighted', zero_division=0) #Weighted F1

print(f"Overall Test Accuracy : {acc:.4f}")
print(f"Weighted Precision   : {prec_w:.4f}")
print(f"Weighted Recall      : {rec_w:.4f}")
print(f"Weighted F1 Score    : {f1_w:.4f}\n")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 952ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Overall Test Accuracy : 0.5198
Weighted Precision   : 0.4513
Weighted Recall      : 0.5198
Weighted F1 Score    : 0.4396

